# Model Context Protocol (MCP) Concepts

The **Model Context Protocol (MCP)** is an open protocol that standardizes how applications provide context to Large Language Models (LLMs). Think of it as a universal adapter that allows AI assistants to connect to various data sources and tools.

> 📝 **Hands-on Exercises**: After reviewing these concepts, complete the exercises in [EXERCISES.md](EXERCISES.md).

## Key Benefits
- **Standardization**: One protocol to connect to many tools
- **Security**: Controlled access to resources
- **Flexibility**: Works with any LLM provider

## MCP Architecture

```
+-----------------+     +-----------------+     +-----------------+
|   AI Agent      |---->|   MCP Client    |---->|   MCP Server    |
|   (Host)        |     |                 |     |   (Tools)       |
+-----------------+     +-----------------+     +-----------------+
```

### Components
1. **Host**: The AI application (e.g., Claude, ChatGPT integration, or custom agent)
2. **Client**: Connects to MCP servers on behalf of the host
3. **Server**: Exposes tools, resources, and prompts

## Transport Types

MCP supports two primary transport mechanisms:

### 1. STDIO (Standard Input/Output)
- Used for **local** MCP servers
- Server runs as a subprocess
- Communication via stdin/stdout
- Best for: Local tools, CLI applications

### 2. HTTP/SSE (Server-Sent Events)
- Used for **remote** MCP servers
- Server runs as a web service
- Communication via HTTP requests and SSE
- Best for: Cloud services, shared tools

## Workshop Projects Overview

| Project | Type | Transport | Description |
|---------|------|-----------|-------------|
| mcp_local_server | Local | STDIO | Python MCP server with Config and Ticket tools |
| mcp_remote_server | REST API | HTTP | Backend REST API for tickets |
| mcp_bridge | Remote | HTTP/SSE | MCP server that calls REST API |
| mcp_agent_client | Client | Both | AI agent that consumes MCP servers |

## Setup: Install Required Packages

Run the following cell to install the required packages for MCP and Azure OpenAI integration.

In [ ]:
# Setup: Install required packages
# Run this cell first to ensure all dependencies are installed

import subprocess
import sys

packages = [
    "mcp",
    "httpx",
    "httpx-sse", 
    "openai",
    "azure-identity",
    "python-dotenv"
]

for package in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
    
print("✅ All packages installed successfully!")

In [ ]:
# Configuration: Load environment variables
# Set up your Azure OpenAI connection

import os
import json
from pathlib import Path
from dotenv import load_dotenv


def find_config_path(start_path: str) -> str:
    """Find the 'python' folder by traversing up from start_path."""
    current_dir = Path(start_path)
    
    while current_dir is not None:
        if current_dir.name.lower() == "python":
            return str(current_dir)
        if current_dir.parent == current_dir:
            break
        current_dir = current_dir.parent
    
    # Fallback to start path if python folder not found
    return start_path


def load_env_file(env_path: str) -> dict:
    """Load environment variables from .env file (JSON format)."""
    env_file = Path(env_path) / ".env"
    
    if not env_file.exists():
        return {}
    
    try:
        with open(env_file, 'r') as f:
            content = f.read()
            env_vars = json.loads(content)
            
            # Set environment variables
            for key, value in env_vars.items():
                os.environ[key] = str(value)
            
            return env_vars
    except json.JSONDecodeError:
        # Fallback: try loading as standard dotenv format
        load_dotenv(env_file, override=True)
        return {}
    except IOError as e:
        print(f"Warning: Failed to load .env file: {e}")
        return {}


# Load environment variables from .env file in the python root folder
config_path = find_config_path(os.getcwd())
env_vars = load_env_file(config_path)
if env_vars:
    print(f"✅ Loaded {len(env_vars)} environment variables from: {config_path}/.env")
else:
    # Fallback: try loading from current directory
    load_dotenv()
    print("⚠️ Loaded .env from current directory (fallback)")

# Check required configuration - support both naming conventions
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT") or os.getenv("AZURE_AI_PROJECT_ENDPOINT")
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME") or os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o-mini")

if endpoint:
    print(f"✅ Azure OpenAI Endpoint: {endpoint}")
    print(f"✅ Deployment Name: {deployment}")
else:
    print("⚠️ Azure endpoint not set")
    print("Please set one of the following environment variables:")
    print("  - AZURE_AI_PROJECT_ENDPOINT or AZURE_OPENAI_ENDPOINT")
    print("And optionally:")
    print("  - AZURE_AI_MODEL_DEPLOYMENT_NAME or AZURE_OPENAI_DEPLOYMENT_NAME (default: gpt-4o-mini)")
    print("  - AZURE_OPENAI_API_KEY (for API key auth)")

# Check authentication method
api_key = os.getenv("AZURE_OPENAI_API_KEY")
tenant_id = os.getenv("AZURE_TENANT_ID")
client_id = os.getenv("AZURE_CLIENT_ID")
client_secret = os.getenv("AZURE_CLIENT_SECRET")

if api_key:
    print("✅ Authentication: API Key")
elif tenant_id and client_id and client_secret:
    print("✅ Authentication: Service Principal")
else:
    print("✅ Authentication: Azure CLI / DefaultAzureCredential")

In [ ]:
# Create Azure OpenAI Client
# This demonstrates how to set up the client with different auth methods

import os
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, AzureCliCredential, ClientSecretCredential, get_bearer_token_provider

# Get configuration - AZURE_OPENAI takes priority over AZURE_AI_PROJECT
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini")

# Only fall back to Foundry endpoint if Azure OpenAI endpoint is not set
if not endpoint:
    endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
    deployment = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", deployment)
    print("⚠️ Using Azure AI Foundry endpoint (AZURE_AI_PROJECT_ENDPOINT)")

if not endpoint:
    raise ValueError("Azure endpoint not set. Set AZURE_OPENAI_ENDPOINT environment variable.")

api_version = "2024-02-15-preview"

# Use API Key if provided (standard Azure OpenAI)
if api_key:
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_key=api_key,
        api_version=api_version
    )
    print(f"✅ Created AzureOpenAI client with API Key auth")
    print(f"   Endpoint: {endpoint}")
    print(f"   Deployment: {deployment}")
else:
    # Token-based auth (for Foundry endpoints or when no API key)
    print("⚠️ No API key found, using token-based authentication")
    try:
        credential = AzureCliCredential()
        token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
        client = AzureOpenAI(
            azure_endpoint=endpoint,
            azure_ad_token_provider=token_provider,
            api_version=api_version
        )
        print("✅ Created AzureOpenAI client with Azure CLI auth")
    except Exception as e:
        credential = DefaultAzureCredential()
        token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")
        client = AzureOpenAI(
            azure_endpoint=endpoint,
            azure_ad_token_provider=token_provider,
            api_version=api_version
        )
        print("✅ Created AzureOpenAI client with DefaultAzureCredential")

In [ ]:
# Debug: Check your configuration values
import os

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT") or os.getenv("AZURE_AI_PROJECT_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME") or os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o-mini")

print("=== Configuration Check ===")
print(f"Endpoint: {endpoint}")
print(f"API Key: {'*' * 10 + api_key[-4:] if api_key and len(api_key) > 4 else 'NOT SET or too short'}")
print(f"Deployment: {deployment}")

# Validate endpoint format
if endpoint:
    if not endpoint.startswith("https://"):
        print("⚠️ Endpoint should start with 'https://'")
    if not "openai.azure.com" in endpoint:
        print("⚠️ Endpoint should contain 'openai.azure.com'")
    if endpoint.endswith("/"):
        print("⚠️ Endpoint should NOT end with '/' - this can cause auth errors")
else:
    print("❌ Endpoint is not set!")

In [ ]:
# Test the Azure OpenAI connection
# Send a simple completion request to verify everything is working

response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Say 'Hello from MCP Lab!' in exactly 5 words."}
    ],
    max_tokens=50
)

print("Response from Azure OpenAI:")
print(f"  {response.choices[0].message.content}")
print()
print(f"✅ Azure OpenAI connection successful!")
print(f"  Model: {response.model}")
print(f"  Tokens: {response.usage.total_tokens}")

In [ ]:
# MCP Server Example - Define tools
# This shows how to create MCP tools

from mcp.server import Server
from mcp.types import Tool, TextContent

# Create an MCP server instance
server = Server("example-server")

# Define a simple tool
@server.list_tools()
async def list_tools() -> list[Tool]:
    """List available tools."""
    return [
        Tool(
            name="GetConfig",
            description="Gets a configuration value by key",
            inputSchema={
                "type": "object",
                "properties": {
                    "key": {
                        "type": "string",
                        "description": "The configuration key"
                    }
                },
                "required": ["key"]
            }
        )
    ]

print("✅ MCP Server defined with tools")
print("Available tools:")
tools = await list_tools()
for tool in tools:
    print(f"  - {tool.name}: {tool.description}")

#  MCP Concepts - Python

This notebook explains the key concepts of the **Model Context Protocol (MCP)** and how to use it with Python.

## Table of Contents

1. [What is MCP?](#what-is-mcp)
2. [MCP Architecture](#mcp-architecture)
3. [Transport Types](#transport-types)
4. [Workshop Projects Overview](#workshop-projects-overview)
5. [Creating an MCP Server](#creating-an-mcp-server)
6. [Creating an MCP Client](#creating-an-mcp-client)
7. [AI Agent Integration](#ai-agent-integration)
8. [HTTP/SSE Transport](#httpsse-transport)
9. [Best Practices](#best-practices)

## What is MCP?

The **Model Context Protocol (MCP)** is an open protocol that standardizes how applications provide context to Large Language Models (LLMs). Think of it as a universal adapter that allows AI assistants to connect to various data sources and tools.

### Key Benefits

| Benefit | Description |
|---------|-------------|
| **Standardization** | One protocol to connect to many tools |
| **Security** | Controlled access to resources |
| **Flexibility** | Works with any LLM provider |
| **Interoperability** | Tools written in any language can be consumed |

## MCP Architecture

`
          
   AI Agent         MCP Client       MCP Server    
   (Host)                                      (Tools)       
          
`

### Components

| Component | Description |
|-----------|-------------|
| **Host** | The AI application (e.g., Claude, ChatGPT integration) |
| **Client** | Connects to MCP servers on behalf of the host |
| **Server** | Exposes tools, resources, and prompts |

## Transport Types

MCP supports two primary transport mechanisms:

### 1. STDIO (Standard Input/Output)
- Used for **local** MCP servers
- Server runs as a subprocess
- Communication via stdin/stdout
- Best for: Local tools, CLI applications

### 2. HTTP/SSE (Server-Sent Events)
- Used for **remote** MCP servers
- Server runs as a web service
- Communication via HTTP requests and SSE
- Best for: Cloud services, shared tools

### When to Use Which?

| Scenario | Transport | Reason |
|----------|-----------|--------|
| Local file access | STDIO | Direct filesystem access |
| Low latency critical | STDIO | No network overhead |
| Remote API access | HTTP/SSE | Network communication |
| Multi-user scenarios | HTTP/SSE | Shared server instance |
| Centralized monitoring | HTTP/SSE | Server-side logging |

## Workshop Projects Overview

| Project | Type | Transport | Description |
|---------|------|-----------|-------------|
| mcp_local_server | Local | STDIO | Python MCP server with Config and Ticket tools |
| mcp_remote_server | REST API | HTTP | FastAPI backend REST API for tickets |
| mcp_bridge | Remote | HTTP/SSE | MCP server that calls REST API |
| mcp_agent_client | Client | Both | AI agent that consumes MCP servers |

## Creating an MCP Server

### Basic Server Structure

`python
from mcp.server import Server
from mcp.server.stdio import stdio_server
import asyncio

# Create server instance
server = Server("mcp-local-server")
`

### Defining Tools with Decorators

`python
@server.tool()
async def get_config() -> str:
    """Gets the current configuration."""
    return "Current config value"

@server.tool()
async def update_config(value: str) -> str:
    """Updates the configuration.
    
    Args:
        value: The new configuration value
    """
    return f"Updated to: {value}"
`

### Tool with Complex Parameters

`python
from typing import Optional

@server.tool()
async def create_ticket(
    subject: str,
    description: str,
    customer_name: str,
    priority: str = "MEDIUM"
) -> str:
    """Creates a new support ticket.
    
    Args:
        subject: Brief summary of the issue
        description: Detailed description of the problem
        customer_name: Name of the customer
        priority: Priority level (LOW, MEDIUM, HIGH)
    """
    ticket_id = str(uuid.uuid4())[:8]
    # Store ticket logic here
    return f"Ticket {ticket_id} created successfully"
`

### Running the Server (STDIO)

`python
async def main():
    async with stdio_server() as (read_stream, write_stream):
        await server.run(
            read_stream,
            write_stream,
            server.create_initialization_options()
        )

if __name__ == "__main__":
    asyncio.run(main())
`

## Creating an MCP Client

### STDIO Client Connection

`python
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

# Define server parameters
server_params = StdioServerParameters(
    command="python",
    args=["-m", "mcp_local_server.main"]
)

# Connect and use
async with stdio_client(server_params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        
        # List available tools
        tools = await session.list_tools()
        for tool in tools.tools:
            print(f"Tool: {tool.name} - {tool.description}")
        
        # Call a tool
        result = await session.call_tool("get_config", {})
        print(f"Result: {result.content[0].text}")
`

### SSE Client Connection (Remote)

`python
from mcp.client.sse import sse_client

async with sse_client("http://localhost:5070/sse") as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        
        # Use remote tools
        result = await session.call_tool("get_all_tickets", {})
        print(result.content[0].text)
`

## AI Agent Integration

The real power of MCP comes when integrating it with AI agents.

### Converting MCP Tools to OpenAI Format

`python
def mcp_tools_to_openai_format(mcp_tools):
    """Convert MCP tools to OpenAI function calling format."""
    openai_tools = []
    for tool in mcp_tools.tools:
        openai_tool = {
            "type": "function",
            "function": {
                "name": tool.name,
                "description": tool.description or "",
                "parameters": tool.inputSchema if tool.inputSchema else {
                    "type": "object",
                    "properties": {},
                    "required": []
                }
            }
        }
        openai_tools.append(openai_tool)
    return openai_tools
`

### Creating the Azure OpenAI Client

`python
import os
from openai import AzureOpenAI
from azure.identity import AzureCliCredential, DefaultAzureCredential

def create_chat_client() -> AzureOpenAI:
    """Create Azure OpenAI client with multiple auth options."""
    endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    
    if api_key:
        return AzureOpenAI(
            azure_endpoint=endpoint,
            api_key=api_key,
            api_version="2024-02-15-preview"
        )
    else:
        credential = DefaultAzureCredential()
        token = credential.get_token("https://cognitiveservices.azure.com/.default")
        return AzureOpenAI(
            azure_endpoint=endpoint,
            api_key=token.token,
            api_version="2024-02-15-preview"
        )
`

### AI Agent with Tool Calling

`python
import json

async def run_agent_with_tools(client, session, user_message: str):
    """Run an AI agent that can use MCP tools."""
    
    # Get MCP tools and convert to OpenAI format
    mcp_tools = await session.list_tools()
    openai_tools = mcp_tools_to_openai_format(mcp_tools)
    
    messages = [
        {"role": "system", "content": "You are a helpful assistant with access to support ticket tools."},
        {"role": "user", "content": user_message}
    ]
    
    # Call the AI model
    response = client.chat.completions.create(
        model=os.environ.get("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o-mini"),
        messages=messages,
        tools=openai_tools,
        tool_choice="auto"
    )
    
    assistant_message = response.choices[0].message
    
    # Handle tool calls
    if assistant_message.tool_calls:
        for tool_call in assistant_message.tool_calls:
            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)
            
            # Execute MCP tool
            result = await session.call_tool(tool_name, tool_args)
            print(f"Tool {tool_name} returned: {result.content[0].text}")
    
    return assistant_message.content
`

## HTTP/SSE Transport

### Server-Side Setup (FastAPI + Starlette)

`python
from mcp.server import Server
from mcp.server.sse import SseServerTransport
from starlette.applications import Starlette
from starlette.routing import Route
import uvicorn

# Create MCP server and SSE transport
server = Server("mcp-bridge-server")
sse = SseServerTransport("/messages")

async def handle_sse(request):
    async with sse.connect_sse(
        request.scope,
        request.receive,
        request._send
    ) as streams:
        await server.run(
            streams[0],
            streams[1],
            server.create_initialization_options()
        )

# Define routes
app = Starlette(
    debug=True,
    routes=[
        Route("/sse", endpoint=handle_sse),
        Route("/messages", endpoint=sse.handle_post_message, methods=["POST"]),
    ]
)

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=5070)
`

### MCP Bridge Pattern

The MCP Bridge pattern wraps a REST API with MCP tools:

`python
import httpx

REST_API_URL = "http://localhost:5060"

@server.tool()
async def get_all_tickets() -> str:
    """Gets all support tickets from the remote API."""
    async with httpx.AsyncClient() as client:
        response = await client.get(f"{REST_API_URL}/tickets")
        response.raise_for_status()
        return response.text

@server.tool()
async def create_ticket(
    subject: str,
    description: str,
    customer_name: str
) -> str:
    """Creates a new ticket via the remote API."""
    async with httpx.AsyncClient() as client:
        response = await client.post(
            f"{REST_API_URL}/tickets",
            json={
                "subject": subject,
                "description": description,
                "customer_name": customer_name
            }
        )
        response.raise_for_status()
        return response.text
`

## Best Practices

### Tool Design

`python
#  Good: Clear docstrings, typed parameters
@server.tool()
async def get_ticket(ticket_id: str) -> str:
    """Retrieves a support ticket by its ID.
    
    Args:
        ticket_id: The unique identifier of the ticket (e.g., 'TKT-001')
    
    Returns:
        JSON string containing ticket details
    """
    # Implementation

#  Bad: No documentation, unclear parameters
@server.tool()
async def get(id):
    # Implementation
`

### Error Handling

`python
@server.tool()
async def get_ticket(ticket_id: str) -> str:
    """Retrieves a support ticket."""
    try:
        ticket = await fetch_ticket(ticket_id)
        if not ticket:
            return json.dumps({"error": f"Ticket {ticket_id} not found"})
        return json.dumps(ticket)
    except Exception as e:
        return json.dumps({"error": str(e)})
`

### Authentication Options

| Method | Environment Variables | Use Case |
|--------|----------------------|----------|
| API Key | AZURE_OPENAI_API_KEY | Development |
| Azure CLI | None (uses az login) | Local development |
| Service Principal | AZURE_TENANT_ID, AZURE_CLIENT_ID, AZURE_CLIENT_SECRET | CI/CD |
| Managed Identity | None (automatic) | Production Azure |

## Running the Workshop

### Step 1: Install Dependencies
`bash
cd python/lab3-mcp
pip install -r requirements.txt
`

### Step 2: Configure Environment
`bash
# Set Azure OpenAI configuration
export AZURE_OPENAI_ENDPOINT="https://your-resource.openai.azure.com"
export AZURE_OPENAI_DEPLOYMENT_NAME="gpt-4o-mini"

# Option A: API Key
export AZURE_OPENAI_API_KEY="your-api-key"

# Option B: Azure CLI (login first)
az login
`

### Step 3: Start the REST API
`bash
python -m mcp_remote_server.main  # Starts on port 5060
`

### Step 4: Start the MCP Bridge (optional)
`bash
python -m mcp_bridge.main  # Starts on port 5070
`

### Step 5: Run the Agent Client
`bash
python -m mcp_agent_client.main
`

## Summary

| Concept | Description |
|---------|-------------|
| **MCP** | Protocol for AI-to-tool communication |
| **STDIO Transport** | Local subprocess communication |
| **HTTP/SSE Transport** | Remote web service communication |
| **Tools** | Functions exposed by MCP servers |
| **@server.tool()** | Decorator to mark a function as an MCP tool |
| **ClientSession** | Client for connecting to MCP servers |
| **Bridge Pattern** | Wrap REST APIs with MCP interface |

### Quick Reference

`python
# Create server
server = Server("my-server")

# Define tool
@server.tool()
async def my_tool(param: str) -> str:
    """Tool description."""
    return "result"

# STDIO client
async with stdio_client(params) as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
        result = await session.call_tool("my_tool", {"param": "value"})

# SSE client
async with sse_client("http://localhost:5070/sse") as (read, write):
    async with ClientSession(read, write) as session:
        await session.initialize()
`